In [3]:
import os, sys
import pandas as pd
import numpy as np
from pprint import pprint
import re

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
sys.path.append('../src')

In [4]:
from download import Config

In [5]:
def convert_to_camelcase(input_string):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', input_string)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()


In [15]:
def check_len_strings(df, check='max_length', print_out=False):
    """print out the max or min length of string columns in the dataframe
    df: pd.DataFrame
    check: 'max_length' or 'min_length' 
    returns pd.Series of column names and max string lengths"""
    
    assert (check=='max_length') or (check=='min_length')
    lens_dict = {}
    
    for col in df.columns:
        ser = df[col]
        
        if ser.dtype == np.object:
            col_lens = df[col].str.len()
            
            if check=='max_length':
                len_val = col_lens.max()
            
            elif check=='min_length':
                len_val = col_lens.min()

            lens_dict[col] = len_val
            
            if print_out:
                print(col,' : ', int(len_val))
    
    return pd.Series(lens_dict, dtype=int, name=check)

In [30]:
# Make series of string length and value
def get_ser_of_extreme_vals(df, agg='max'):
    """
    df: pandasdataframe
    agg: 'max' or 'min' check for max or min length of strings and values of numerics
    
    returns pd.Series
    """

    ser_lens = check_len_strings(df, check=f'{agg}_length')
    fltr = [pd.api.types.is_numeric_dtype(df[col]) for col in df.columns]
    ser_vals = df.loc[:, fltr]
    if agg =='max':
        ser_vals = ser_vals.max()
    elif agg =='min':
        ser_vals = ser_vals.min()

    ser_vals = pd.concat([ser_vals,  ser_lens])
    ser_vals.name = f'{agg}_vals'

    return ser_vals

In [37]:
def show_df_dtypes_and_vals(df):

    # make dataframe of datatypes
    ser_types = df.dtypes
    ser_types.name = 'datatypes'
    df_summary = pd.DataFrame(ser_types)

    #add extreme values
    ser_min_vals = get_ser_of_extreme_vals(df, agg='min')
    ser_max_vals = get_ser_of_extreme_vals(df, agg='max')
    df_summary = df_summary.join(ser_min_vals, how='left').join(ser_max_vals, how='left')
    # reorder to original column order
    df_summary = df_summary.loc[df.columns,:]

    return df_summary

In [43]:
docid = Config.googlesheet_uids['resources']
url = f"https://docs.google.com/spreadsheets/d/{docid}"
export_url = url + "/export?format=xlsx"
url

'https://docs.google.com/spreadsheets/d/1LeLlrsvBWMYTTIXTVtkynmBzzb0Uzi1OwpRLfyRAwzM'

In [8]:
fldr_path = '..','data','raw'
fname = 'assessments.xlsx'
fpath = os.path.join(*fldr_path, fname)

In [9]:
xl = pd.ExcelFile(fpath)
sheet_names = xl.sheet_names  # see all sheet names

In [10]:
cols_dict = {}

for sheet_name in sheet_names:
    df = xl.parse(sheet_name, nrows=0)
    cols = df.columns.tolist()
    cols_dict[sheet_name] = cols



In [11]:
for key, col_list in cols_dict.items():
    print('-'*50,'\n', key)
    for col in col_list:
        print(' '*4, col, ' '*4, )
        

-------------------------------------------------- 
 Classes
     ClassName     
     label     
     definition     
     sameAs     
     equivalentClasses     
     equivalentClasses?     
     subClassOf     
-------------------------------------------------- 
 Properties
     property     
     label     
     definition     
     sameAs     
     equivalentProperty     
     equivalentProperty?     
     subPropertyOf     
     propertyDomain     
     propertyRange     
-------------------------------------------------- 
 questionnaires
     index     
     title     
     link     
     description     
     abbreviation     
     last_modified     
     last_modified_by     
     authors     
     year     
     use_with_assessments     
     indices_reference     
     ANIRUDH: LEFT column     
     index_license     
     index_language     
     indices_language_not_in_mhdb     
     LEFT column errors     
     indices_respondent     
     indices_subject     
     age_min

In [38]:
df = xl.parse('references')
df.head()

,index,title,link,authors,entry_date,last_modified_by,pubdate,PubMedID,cogatlas_node_id,cogatlas_prop_id
0,1,DSM-5 Autism Spectrum Disorder: Guidelines and...,https://depts.washington.edu/dbpeds/Screening%...,"Carpenter, L",9 jan 2020,Alessandra Badioli,2013,NaN,NaN,NaN
1,2,CDC Obsessive-Compulsive Disorder,https://www.cdc.gov/childrensmentalhealth/ocd....,NaN,9 jan 2020,Alessandra Badioli,2019 Mar 12,NaN,NaN,NaN
2,4,University of Rochester / Encyclopedia / Separ...,https://www.urmc.rochester.edu/encyclopedia/co...,NaN,9 jan 2020,Alessandra Badioli,NaN,NaN,NaN,NaN
3,5,National Institute of Mental health: Social An...,https://www.nimh.nih.gov/health/publications/s...,NaN,9 jan 2020,Alessandra Badioli,NaN,NaN,NaN,NaN
4,6,National Institute of Mental health: Anxiety D...,https://www.nimh.nih.gov/health/topics/anxiety...,NaN,9 jan 2020,Alessandra Badioli,2018 Jul,NaN,NaN,NaN


In [39]:
show_df_dtypes_and_vals(df)

,datatypes,min_vals,max_vals
index,int64,1.0,2092.0
title,object,13.0,230.0
link,object,28.0,246.0
authors,object,6.0,336.0
entry_date,object,10.0,12.0
last_modified_by,object,3.0,18.0
pubdate,object,4.0,14.0
PubMedID,float64,514759.0,30577944.0
cogatlas_node_id,float64,30412.0,49716.0
cogatlas_prop_id,object,17.0,17.0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1181 entries, 0 to 1180
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             1181 non-null   int64  
 1   title             1181 non-null   object 
 2   link              1178 non-null   object 
 3   authors           1166 non-null   object 
 4   entry_date        1181 non-null   object 
 5   last_modified_by  1181 non-null   object 
 6   pubdate           1166 non-null   object 
 7   PubMedID          1100 non-null   float64
 8   cogatlas_node_id  1086 non-null   float64
 9   cogatlas_prop_id  7 non-null      object 
dtypes: float64(2), int64(1), object(7)
memory usage: 92.4+ KB


In [40]:
## Describe the numerical values
fltr = [pd.api.types.is_numeric_dtype(df[col]) for col in df.columns]
df.loc[:, fltr].describe()

,index,PubMedID,cogatlas_node_id
count,1181.000000,1.100000e+03,1086.000000
mean,1436.108383,1.560890e+07,31427.008287
std,472.577437,6.220990e+06,2138.476465
min,1.000000,5.147590e+05,30412.000000
25%,1201.000000,1.100471e+07,30698.250000
50%,1496.000000,1.668111e+07,30980.500000
75%,1791.000000,2.068678e+07,31326.500000
max,2092.000000,3.057794e+07,49716.000000
